In [ ]:
import cv2
import glob
import numpy as np
from keras.preprocessing.image import random_rotation

kusa_X_train = []

for f in glob.glob("image(きのこ)/train/クサウラベニタケ/*.jpg"):

  img=cv2.imread(f)/255.0
  #img=cv2.imread(f)
  img_resized = cv2.resize(img, (100, 100) )#resizeは横縦の順番で指定するので注意
  kusa_X_train.append(img_resized)               # numpy型データをリストに追加

  for i in range(9):
      arr2 = img_resized
      arr2 = random_rotation(arr2, rg=360)
      kusa_X_train.append(arr2) 


kusa_X_train= np.array(kusa_X_train)

In [ ]:
import cv2
import glob
import numpy as np

ura_X_train = []

for f in glob.glob("image(きのこ)/train/ウラベニホテイシメジ/*.jpg"):

  img=cv2.imread(f)/255.0
  img_resized = cv2.resize(img, (100, 100) )#resizeは横縦の順番で指定するので注意
  ura_X_train.append(img_resized)               # numpy型データをリストに追加

  for i in range(9):
      arr2 = img_resized
      arr2 = random_rotation(arr2, rg=360)
      ura_X_train.append(arr2) 


ura_X_train= np.array(ura_X_train)

In [ ]:
X_train = np.concatenate([kusa_X_train, ura_X_train], axis=0)
y_train = np.concatenate([np.ones((len(kusa_X_train),1)), np.zeros((len(ura_X_train), 1))], axis=0)#クサウラベニタケだと1,ウラベニホテイシメジだと0と指定

In [ ]:
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential
from keras.applications.densenet import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping

# Densenetのapi(https://keras.io/ja/applications/#densenet)
model = Sequential()
model.add(DenseNet121(weights='imagenet', include_top=False, input_shape=(100,100,3)))
model.add(MaxPooling2D())
model.add(Dense(100, activation='ReLU'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])#学習モデル精度の設定(このように学習してくださいという指定)

es = EarlyStopping(monitor='val_loss', patience=5)

model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=30,
    callbacks=[es],
    validation_split=0.2
)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_true=y_test, y_score=y_pred[:, 0, 0, 1])

plt.plot(fpr, tpr, label='roc curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], linestyle='--', label='random')
plt.plot([0, 0, 1], [0, 1, 1], linestyle='--', label='ideal')
plt.legend()
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()